In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [2]:
df=pd.read_csv("RMS_Crime_Incidents.csv")
df.head()

C:\Users\wasif\AppData\Local\Temp\ipykernel_10360\1431475646.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("RMS_Crime_Incidents.csv")


,X,Y,incident_entry_id,nearest_intersection,offense_category,offense_description,state_offense_code,arrest_charge,charge_description,incident_occurred_at,...,report_number,scout_car_area,police_precinct,census_block_2020_geoid,neighborhood,council_district,zip_code,longitude,latitude,ESRI_OID
0,-82.925829,42.411686,567921-5201,Hereford St & Southampton St,WEAPONS OFFENSES,WEAPONS OFFENSE - CONCEALED,5201,52001,WEAPONS OFFENSE - CONCEALED,2020/08/20 18:12:00+00,...,2008200208,0505,05,2.616350e+14,Cornerstone Village,4.0,48224.0,-82.925829,42.411686,1
1,-83.220482,42.334485,510658-1301,Faust Ave & Dayton St,ASSAULT,ASSAULT AND BATTERY/SIMPLE ASSAULT,1301,13001,ASSAULT AND BATTERY/SIMPLE ASSAULT,2020/03/18 18:28:00+00,...,2003180193,0612,06,2.616355e+14,Warrendale,7.0,48228.0,-83.220482,42.334485,2
2,-83.160724,42.424184,567826-2099,Curtis St & Wyoming St,ARSON,ARSON,2099,20000,ARSON,2020/08/20 09:07:00+00,...,2008200067,1207,12,2.616354e+14,Schulze,2.0,48221.0,-83.160724,42.424184,3
3,-83.247337,42.426946,588395-2900,Clarita St & Pierson St,DAMAGE TO PROPERTY,DAMAGE TO PROPERTY,2900,29000,DAMAGE TO PROPERTY,2020/10/10 14:20:00+00,...,2010100129,0806,08,2.616354e+14,Holcomb Community,1.0,48219.0,-83.247337,42.426946,4
4,-83.168461,42.380601,567931-5201,Meyers Rd & Grand River Ave,WEAPONS OFFENSES,WEAPONS OFFENSE - CONCEALED,5201,52001,WEAPONS OFFENSE - CONCEALED,2020/08/20 17:00:00+00,...,2008200176,0205,02,2.616354e+14,Pride Area Community,7.0,48227.0,-83.168461,42.380601,5


In [3]:
df.shape

(710841, 30)

In [4]:
df.columns

Index(['X', 'Y', 'incident_entry_id', 'nearest_intersection',
       'offense_category', 'offense_description', 'state_offense_code',
       'arrest_charge', 'charge_description', 'incident_occurred_at',
       'incident_time', 'incident_day_of_week', 'incident_hour_of_day',
       'incident_year', 'case_id', 'case_status', 'case_status_updated_at',
       'updated_in_ibr_at', 'updated_at', 'crime_id', 'report_number',
       'scout_car_area', 'police_precinct', 'census_block_2020_geoid',
       'neighborhood', 'council_district', 'zip_code', 'longitude', 'latitude',
       'ESRI_OID'],
      dtype='object')

In [5]:
df.describe()

,X,Y,state_offense_code,incident_day_of_week,incident_hour_of_day,incident_year,case_id,crime_id,report_number,census_block_2020_geoid,council_district,zip_code,longitude,latitude,ESRI_OID
count,710023.000000,710023.000000,710841.000000,710841.000000,710841.000000,710841.000000,7.108410e+05,7.108410e+05,7.108410e+05,7.080200e+05,703692.000000,704550.000000,710023.000000,710023.000000,710841.000000
mean,-83.107795,42.389907,2395.476600,4.011282,12.611049,2020.667032,6.090257e+05,3.609026e+06,2.078327e+09,2.616353e+14,3.998938,48219.004606,-83.107795,42.389907,355421.000000
std,0.097516,0.038121,1145.126669,1.992073,7.110615,2.575415,3.401860e+05,3.401860e+05,2.465433e+08,3.856698e+08,2.027061,11.357270,0.097516,0.038121,205202.265688
min,-83.311156,42.225981,901.000000,1.000000,0.000000,1915.000000,4.000000e+00,3.000004e+06,4.000000e+00,2.616350e+14,1.000000,48201.000000,-83.311156,42.225981,1.000000
25%,-83.192954,42.359200,1302.000000,2.000000,7.000000,2019.000000,3.176620e+05,3.317662e+06,1.901290e+09,2.616351e+14,2.000000,48208.000000,-83.192954,42.359200,177711.000000
50%,-83.116714,42.394622,2306.000000,4.000000,14.000000,2021.000000,6.169540e+05,3.616954e+06,2.104100e+09,2.616353e+14,4.000000,48219.000000,-83.116714,42.394622,355421.000000
75%,-83.027513,42.422648,2900.000000,6.000000,19.000000,2023.000000,9.038460e+05,3.903846e+06,2.305230e+09,2.616354e+14,6.000000,48228.000000,-83.027513,42.422648,533131.000000
max,-82.880609,42.470321,9915.000000,7.000000,23.000000,2025.000000,1.187525e+06,4.187525e+06,3.303060e+09,2.616399e+14,7.000000,48243.000000,-82.880609,42.470321,710841.000000


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 710841 entries, 0 to 710840
Data columns (total 30 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   X                        710023 non-null  float64
 1   Y                        710023 non-null  float64
 2   incident_entry_id        710841 non-null  object 
 3   nearest_intersection     710023 non-null  object 
 4   offense_category         710841 non-null  object 
 5   offense_description      710841 non-null  object 
 6   state_offense_code       710841 non-null  int64  
 7   arrest_charge            710841 non-null  object 
 8   charge_description       710841 non-null  object 
 9   incident_occurred_at     710841 non-null  object 
 10  incident_time            710841 non-null  object 
 11  incident_day_of_week     710841 non-null  int64  
 12  incident_hour_of_day     710841 non-null  int64  
 13  incident_year            710841 non-null  int64  
 14  case

In [7]:
#Paper ke mutabiq humein sirf yeh columns chahiye the kyunke yeh features use hote hain crime type aur location predict karne ke liye.
df = df[['incident_day_of_week', 'incident_hour_of_day', 'zip_code', 'offense_category']]

# Rename columns just for our understanding according to paper bcz these column names are mentioned in the paper
df = df.rename(columns={
    'incident_day_of_week': 'day_of_week',
    'incident_hour_of_day': 'hour',
    'offense_category': 'crime_category'
})

In [8]:
# Check for missing values and drop them
df = df.dropna()
print("Missing values after drop:\n", df.isnull().sum())

Missing values after drop:
 day_of_week       0
hour              0
zip_code          0
crime_category    0
dtype: int64


In [9]:
# Count incidents per crime category
category_counts = df['crime_category'].value_counts()
category_counts

crime_category
ASSAULT                   129478
LARCENY                   104131
DAMAGE TO PROPERTY         92003
AGGRAVATED ASSAULT         70333
STOLEN VEHICLE             63246
FRAUD                      59240
BURGLARY                   51867
WEAPONS OFFENSES           26099
STOLEN PROPERTY            15547
ROBBERY                    15109
DANGEROUS DRUGS            12177
SEX OFFENSES                9150
OBSTRUCTING JUDICIARY       8971
OBSTRUCTING THE POLICE      8753
SEXUAL ASSAULT              6093
ARSON                       5287
RUNAWAY                     4780
FAMILY OFFENSE              4095
OUIL                        4088
OTHER                       2515
MISCELLANEOUS               2369
HOMICIDE                    2300
DISORDERLY CONDUCT          2264
FORGERY                     1812
KIDNAPPING                  1573
LIQUOR                       582
EXTORTION                    443
JUSTIFIABLE HOMICIDE         187
SOLICITATION                  45
GAMBLING                    

In [10]:
#Dataset mein originally ~38 crime categories thi, lekin paper ke according humne sirf un categories ko rakha jin ke incidents ≥600 the, taake data balanced ho.
selected_categories = category_counts[category_counts >= 600].index
df_filtered = df[df['crime_category'].isin(selected_categories)]
# Verify the number of categories
print(f"Number of crime categories after filtering: {len(selected_categories)}")
print("Selected categories:\n", selected_categories)

print(f"Unique zip codes: {len(df_filtered['zip_code'].unique())}")
print(f"Unique days of week: {len(df_filtered['day_of_week'].unique())}")
print(f"HourHour is numerical: 1 feature")

Number of crime categories after filtering: 25
Selected categories:
 Index(['ASSAULT', 'LARCENY', 'DAMAGE TO PROPERTY', 'AGGRAVATED ASSAULT',
       'STOLEN VEHICLE', 'FRAUD', 'BURGLARY', 'WEAPONS OFFENSES',
       'STOLEN PROPERTY', 'ROBBERY', 'DANGEROUS DRUGS', 'SEX OFFENSES',
       'OBSTRUCTING JUDICIARY', 'OBSTRUCTING THE POLICE', 'SEXUAL ASSAULT',
       'ARSON', 'RUNAWAY', 'FAMILY OFFENSE', 'OUIL', 'OTHER', 'MISCELLANEOUS',
       'HOMICIDE', 'DISORDERLY CONDUCT', 'FORGERY', 'KIDNAPPING'],
      dtype='object', name='crime_category')
Unique zip codes: 30
Unique days of week: 7
HourHour is numerical: 1 feature


In [11]:
# Humne dataset ko do parts mein divide kiya: training set aur validation set.
# Training set ko balanced banaya:
# Har crime category ke liye:
# Agar total incidents ≥700 the, to 350 samples liye.
# Agar 600 ≤ incidents < 700 the, to total / 2 samples liye.
# Badi categories ke liye thodi extra samples add kiye (max 370 tak).
# Baaki sab samples validation set mein gaye.
train_samples = []
for category in selected_categories:
    category_df = df_filtered[df_filtered['crime_category'] == category]
    total = len(category_df)
    if total >= 700:
        n_train = 350
    else:
        n_train = total // 2
    # Add proportional extra samples for large clusters (up to 370)
    if total > 700:
        n_train = min(370, n_train + int((total - 700) * 0.1))
    train_sample = category_df.sample(n=n_train, random_state=42)
    train_samples.append(train_sample)

# Combine training samples
train_df = pd.concat(train_samples)
# Validation set
validation_df = df_filtered[~df_filtered.index.isin(train_df.index)]

# Verify sizes
print(f"Training set size: {len(train_df)}")
print(f"Validation set size: {len(validation_df)}")
# Paper ke according training set balanced hona chahiye taake har category ka equal representation ho.
# Validation set bada tha kyunke woh real-world testing ke liye use hota hai.

Training set size: 9250
Validation set size: 694030


In [12]:
#prepare dataset for our two models as menioned in paper
#1. Crime Type Prediction
#2. Crime Location Prediction

# Categorical variables (zip_code, day_of_week, crime_category) ko one-hot encoding se convert kiya (0s aur 1s ke matrices) or hour ko numerical he rakha
# Reason: Paper ke mutabiq, categorical data ko one-hot encode karna tha kyunke neural networks raw categories nahi samajhte.
zip_encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
day_encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
category_encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')

In [13]:
# Fit encoders
zip_encoder.fit(df_filtered[['zip_code']])
day_encoder.fit(df_filtered[['day_of_week']])
category_encoder.fit(df_filtered[['crime_category']])

E:\BSAI-5th\DataMining\.venv\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
E:\BSAI-5th\DataMining\.venv\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
E:\BSAI-5th\DataMining\.venv\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


OneHotEncoder(handle_unknown='ignore', sparse=False, sparse_output=False)

In [14]:
# Prepare inputs and outputs for crime type prediction
X_train_crime = np.hstack([
    zip_encoder.transform(train_df[['zip_code']]),
    day_encoder.transform(train_df[['day_of_week']]),
    train_df['hour'].values.reshape(-1, 1)
])
y_train_crime = np.argmax(category_encoder.transform(train_df[['crime_category']]), axis=1)
X_val_crime = np.hstack([
    zip_encoder.transform(validation_df[['zip_code']]),
    day_encoder.transform(validation_df[['day_of_week']]),
    validation_df['hour'].values.reshape(-1, 1)
])
y_val_crime = np.argmax(category_encoder.transform(validation_df[['crime_category']]), axis=1)

In [15]:
# Prepare inputs and outputs for location prediction
X_train_loc = np.hstack([
    category_encoder.transform(train_df[['crime_category']]),
    day_encoder.transform(train_df[['day_of_week']]),
    train_df['hour'].values.reshape(-1, 1)
])
y_train_loc = np.argmax(zip_encoder.transform(train_df[['zip_code']]), axis=1)
X_val_loc = np.hstack([
    category_encoder.transform(validation_df[['crime_category']]),
    day_encoder.transform(validation_df[['day_of_week']]),
    validation_df['hour'].values.reshape(-1, 1)
])
y_val_loc = np.argmax(zip_encoder.transform(validation_df[['zip_code']]), axis=1)

In [16]:
# Data ko PyTorch tensors mein convert kiya taake model training ke liye ready ho.
X_train_crime = torch.tensor(X_train_crime, dtype=torch.float32)
y_train_crime = torch.tensor(y_train_crime, dtype=torch.long)
X_val_crime = torch.tensor(X_val_crime, dtype=torch.float32)
y_val_crime = torch.tensor(y_val_crime, dtype=torch.long)

X_train_loc = torch.tensor(X_train_loc, dtype=torch.float32)
y_train_loc = torch.tensor(y_train_loc, dtype=torch.long)
X_val_loc = torch.tensor(X_val_loc, dtype=torch.float32)
y_val_loc = torch.tensor(y_val_loc, dtype=torch.long)

In [17]:
# Verify shapes
print("Crime Type - X_train shape:", X_train_crime.shape, "y_train shape:", y_train_crime.shape)
print("Location - X_train shape:", X_train_loc.shape, "y_train shape:", y_train_loc.shape)

Crime Type - X_train shape: torch.Size([9250, 38]) y_train shape: torch.Size([9250])
Location - X_train shape: torch.Size([9250, 33]) y_train shape: torch.Size([9250])


In [18]:
#Humne do models banaye, dono Multi-Layer Perceptrons (MLPs), jo paper ke according hain. Training ke liye PyTorch use kiya aur Adam optimizer (modern backpropagation) ke sath early stopping add kiya.
# Define the MLP model for Crime Type Prediction
class CrimeTypeMLP(nn.Module):
    def __init__(self):
        super(CrimeTypeMLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(38, 12),
            nn.Sigmoid(),
            nn.Linear(12, 4),
            nn.Sigmoid(),
            nn.Linear(4, 25)
        )
    def forward(self, x):
        return self.layers(x)

In [19]:
# Create data loaders
train_dataset_crime = TensorDataset(X_train_crime, y_train_crime)
val_dataset_crime = TensorDataset(X_val_crime, y_val_crime)
train_loader_crime = DataLoader(train_dataset_crime, batch_size=32, shuffle=True)
val_loader_crime = DataLoader(val_dataset_crime, batch_size=32)

In [20]:
# Initialize model, loss, and optimizer
model_crime = CrimeTypeMLP()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_crime.parameters())

In [21]:
# Training loop with early stopping
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=100, patience=10):
    best_val_loss = float('inf')
    patience_counter = 0
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        # Validation
        model.eval()
        val_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total += y_batch.size(0)
                correct += (predicted == y_batch).sum().item()

        val_loss /= len(val_loader)
        val_accuracy = correct / total
        print(f"Epoch {epoch+1}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy*100:.1f}%")

        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(model.state_dict(), 'crime_type_model.pt')
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered")
                break

# Train the model
train_model(model_crime, train_loader_crime, val_loader_crime, criterion, optimizer)

Epoch 1, Val Loss: 3.3064, Val Accuracy: 1.2%
Epoch 2, Val Loss: 3.2481, Val Accuracy: 1.2%
Epoch 3, Val Loss: 3.2342, Val Accuracy: 0.6%
Epoch 4, Val Loss: 3.2223, Val Accuracy: 3.9%
Epoch 5, Val Loss: 3.2152, Val Accuracy: 6.4%
Epoch 6, Val Loss: 3.2160, Val Accuracy: 6.8%
Epoch 7, Val Loss: 3.2084, Val Accuracy: 7.9%
Epoch 8, Val Loss: 3.2164, Val Accuracy: 6.9%
Epoch 9, Val Loss: 3.2152, Val Accuracy: 7.7%
Epoch 10, Val Loss: 3.2198, Val Accuracy: 3.7%
Epoch 11, Val Loss: 3.2182, Val Accuracy: 4.9%
Epoch 12, Val Loss: 3.2016, Val Accuracy: 8.2%
Epoch 13, Val Loss: 3.2157, Val Accuracy: 2.6%
Epoch 14, Val Loss: 3.2156, Val Accuracy: 1.6%
Epoch 15, Val Loss: 3.2163, Val Accuracy: 1.3%
Epoch 16, Val Loss: 3.2156, Val Accuracy: 2.2%
Epoch 17, Val Loss: 3.2123, Val Accuracy: 1.8%
Epoch 18, Val Loss: 3.2125, Val Accuracy: 1.5%
Epoch 19, Val Loss: 3.2141, Val Accuracy: 2.2%
Epoch 20, Val Loss: 3.2055, Val Accuracy: 1.7%
Epoch 21, Val Loss: 3.2024, Val Accuracy: 1.6%
Epoch 22, Val Loss: 3.

In [22]:
# Load best model and evaluate
model_crime.load_state_dict(torch.load('crime_type_model.pt'))
model_crime.eval()
correct = 0
total = 0
with torch.no_grad():
    for X_batch, y_batch in val_loader_crime:
        outputs = model_crime(X_batch)
        _, predicted = torch.max(outputs, 1)
        total += y_batch.size(0)
        correct += (predicted == y_batch).sum().item()
accuracy = correct / total
print(f"Crime Type Prediction Accuracy: {accuracy * 100:.1f}%")

C:\Users\wasif\AppData\Local\Temp\ipykernel_10360\2823806501.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_crime.load_state_dict(torch.load('crime_type_model.pt'

Crime Type Prediction Accuracy: 8.2%


In [23]:
#Unique crime categories: 25
# Unique days of week: 7
# Hour is numerical: 1 feature
# Total input features for Location Model: 25+7+1=33
# Unique zip codes (output): 30

# Define the MLP model Crime Location Prediction
class LocationMLP(nn.Module):
    def __init__(self):
        super(LocationMLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(33, 27),
            nn.Sigmoid(),
            nn.Linear(27, 6),
            nn.Sigmoid(),
            nn.Linear(6, 30)
        )

    def forward(self, x):
        return self.layers(x)

In [24]:
# Create data loaders
train_dataset_loc = TensorDataset(X_train_loc, y_train_loc)
val_dataset_loc = TensorDataset(X_val_loc, y_val_loc)
train_loader_loc = DataLoader(train_dataset_loc, batch_size=32, shuffle=True)
val_loader_loc = DataLoader(val_dataset_loc, batch_size=32)

In [25]:
# Initialize model, loss, and optimizer
model_loc = LocationMLP()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_loc.parameters())

In [26]:
# Train the model
train_model(model_loc, train_loader_loc, val_loader_loc, criterion, optimizer)

Epoch 1, Val Loss: 3.2146, Val Accuracy: 8.1%
Epoch 2, Val Loss: 3.1768, Val Accuracy: 8.1%
Epoch 3, Val Loss: 3.1752, Val Accuracy: 8.1%
Epoch 4, Val Loss: 3.1757, Val Accuracy: 8.1%
Epoch 5, Val Loss: 3.1758, Val Accuracy: 8.1%
Epoch 6, Val Loss: 3.1760, Val Accuracy: 7.9%
Epoch 7, Val Loss: 3.1751, Val Accuracy: 8.1%
Epoch 8, Val Loss: 3.1761, Val Accuracy: 7.8%
Epoch 9, Val Loss: 3.1755, Val Accuracy: 7.2%
Epoch 10, Val Loss: 3.1751, Val Accuracy: 8.1%
Epoch 11, Val Loss: 3.1749, Val Accuracy: 8.1%
Epoch 12, Val Loss: 3.1756, Val Accuracy: 7.6%
Epoch 13, Val Loss: 3.1740, Val Accuracy: 8.0%
Epoch 14, Val Loss: 3.1735, Val Accuracy: 8.0%
Epoch 15, Val Loss: 3.1735, Val Accuracy: 8.1%
Epoch 16, Val Loss: 3.1710, Val Accuracy: 8.1%
Epoch 17, Val Loss: 3.1711, Val Accuracy: 8.1%
Epoch 18, Val Loss: 3.1702, Val Accuracy: 8.0%
Epoch 19, Val Loss: 3.1695, Val Accuracy: 7.9%
Epoch 20, Val Loss: 3.1689, Val Accuracy: 8.0%
Epoch 21, Val Loss: 3.1674, Val Accuracy: 7.9%
Epoch 22, Val Loss: 3.

In [29]:
# Load best model and evaluate
model_loc.load_state_dict(torch.load('location_model.pt'))
model_loc.eval()
correct = 0
total = 0
with torch.no_grad():
    for X_batch, y_batch in val_loader_loc:
        outputs = model_loc(X_batch)
        _, predicted = torch.max(outputs, 1)
        total += y_batch.size(0)
        correct += (predicted == y_batch).sum().item()
accuracy = correct / total
print(f"Location Prediction Exact Accuracy: {accuracy * 100:.1f}%")

C:\Users\wasif\AppData\Local\Temp\ipykernel_10360\3496721207.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_loc.load_state_dict(torch.load('location_model.pt'))


Location Prediction Exact Accuracy: 8.1%


In [30]:
# Predict zip codes
model_loc.eval()
y_pred_loc = []
y_true_loc = []
with torch.no_grad():
    for X_batch, y_batch in val_loader_loc:
        outputs = model_loc(X_batch)
        _, predicted = torch.max(outputs, 1)
        y_pred_loc.extend(predicted.numpy())
        y_true_loc.extend(y_batch.numpy())

y_pred_loc = np.array(y_pred_loc)
y_true_loc = np.array(y_true_loc)

# Exact accuracy
exact_matches = np.sum(y_pred_loc == y_true_loc)
exact_accuracy = exact_matches / len(y_true_loc)
print(f"Exact Accuracy: {exact_accuracy * 100:.1f}%")

Exact Accuracy: 8.1%


In [28]:
# Save models
torch.save(model_crime.state_dict(), 'crime_type_model.pt')
torch.save(model_loc.state_dict(), 'location_model.pt')